In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import glob
import os
import sys
import json
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import lib.dataset as dataset
import lib.network as network
import lib.utils as utils
import lib.vis as vis
from PIL import Image
from collections import deque
from tensorflow.python.tools import inspect_checkpoint
from moviepy.editor import *
params=utils.read_params()
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

Using TensorFlow backend.


In [24]:
models_remote=utils.list_folders(params["DIRS"]["MODELS_REMOTE"])
window_width=1

raw_experiment_params=pd.DataFrame()
raw_experiment_data=pd.DataFrame()
for i,m in enumerate(models_remote):
    
    model_params=utils.get_model_params(m)
    val_loss=utils.get_latest_loss(m,"val")


#     if model_params["TRAIN"]["EPOCH_COUNT"]-1==utils.get_latest_epoch_index(m):
    raw_experiment_params=raw_experiment_params.append(model_params["TRAIN"],ignore_index=True) 
    raw_experiment_data=raw_experiment_data.append(pd.Series([val_loss.mean()]),ignore_index=True)
        
    
raw_experiment_data=raw_experiment_data.round(3)

In [25]:
# fix table
experiments=pd.DataFrame()
experiments["OPTIMIZER"]=raw_experiment_params["OPTIMIZER"].fillna("SGD")
experiments["LEARN RATE"] = raw_experiment_params["ADAM_LEARN_RATE"]
experiments["LEARN RATE"].fillna(raw_experiment_params["GD_LEARN_RATE"],inplace=True)
experiments["LEARN RATE"].fillna(raw_experiment_params["LEARN_RATE"],inplace=True)
experiments["NETWORK TYPE"]=raw_experiment_params["ENCODER_MODE"]+"-"+raw_experiment_params["RNN_MODE"]+"-"+raw_experiment_params["DECODER_MODE"]
experiments["BATCH SIZE"]=raw_experiment_params["BATCH_SIZE"]
# experiments["EPOCH SIZE"]=raw_experiment_params["EPOCH_COUNT"]
raw_experiment_data.columns=["MEAN VAL LOSS"]
experiments=pd.concat([experiments,raw_experiment_data],axis=1)
experiments.to_csv("experiments.csv",index=False)

In [26]:
experiments

,OPTIMIZER,LEARN RATE,NETWORK TYPE,BATCH SIZE,MEAN VAL LOSS
0,SGD,0.10000,SIMPLE-GRU-SIMPLE,16.0,0.185
1,SGD,0.10000,SIMPLE-GRU-SIMPLE,16.0,0.112
2,SGD,0.10000,SIMPLE-GRU-SIMPLE,16.0,0.155
3,SGD,0.10000,SIMPLE-GRU-SIMPLE,16.0,0.098
4,ADAM,0.00010,SIMPLE-GRU-SIMPLE,16.0,0.112
5,SGD,0.10000,SIMPLE-GRU-SIMPLE,16.0,0.124
6,SGD,0.10000,SIMPLE-GRU-SIMPLE,16.0,0.139
7,ADAM,0.00001,RESIDUAL-GRU-RESIDUAL,8.0,0.153
8,ADAM,0.00001,DILATED-GRU-DILATED,16.0,0.169
9,ADAM,0.00001,RESIDUAL-GRU-RESIDUAL,8.0,0.121


In [ ]:
utils.fix_nparray(utils.get_latest_epoch(models_remote[6])+"/train_loss.npy")

In [ ]:
model_dir=params["SESSIONS"]["LONGEST"]

In [ ]:
net=network.Network_restored(model_dir)

In [ ]:
ls=net.get_operations()

In [ ]:
for i,l in enumerate(ls):
    if "clip" in l.name:
        try:
            print(len(l.inputs[0].shape))
        except:
            print(i)

In [ ]:
x,y=dataset.load_random_sample()
vis.multichannel(x[0])
out=net.predict(x)
vis.voxel(out[0,:,:,:,0],out[0,:,:,:,1],f_name="test.png")
vis.voxel_binary(out[0])

In [ ]:
ims=dataset.load_imgs_from_dir("data/demo_imgs")
ims=np.expand_dims(np.pad(ims,((0,0),(5,5),(5,5),(1,0)),"constant"),0)

In [ ]:

for angle in range(0, 360,30):
    vis.voxel_binary(out[0],view=(30,angle))
    plt.pause(.001)


In [ ]:
op_list=net.get_operations()

In [ ]:
for o in op_list:
    print(o.name)

In [ ]:
f=utils.list_folders("./models_remote")[-1]

In [ ]:
p=glob.glob(f+"/val/event*")[0]
p


In [ ]:
for i in tf.train.summary_iterator(p):
    print(i)
    break

In [ ]:
a=tf.random_uniform([4,4])
a_1=tf.pad(a,[[0,0],[0,0]])
b=tf.random_uniform([8,8])
c=tf.random_uniform([16,16])
d=tf.random_uniform([32,32])

In [ ]:
print(a_1)

In [ ]:
params=utils.read_params()
out_dir=params["DIRS"]["OUTPUT"]

In [ ]:
clip=ImageSequenceClip(dataset.construct_file_path_list_from_dir("/Users/micmelesse/Documents/3D-reconstruction-with-neural-networks/output/02691156_131db4a650873babad3ab188d086d4db",".png"),fps=12)
clip.write_videofile("{}/movie.mp4".format(out_dir))

In [ ]:
out_dir

In [ ]:
epoch_dir="models_remote/model_2018-04-02_20:10:12_L:0.1_E:40_B:16/epoch_0"
model_dir=os.path.dirname(epoch_dir)
X_test,y_test=dataset.load_testset(epoch_dir)
model_info = utils.get_model_info(model_dir)

In [ ]:
x,y=dataset.load_random_sample()
vis.sample(x,y,y,"test.png")

In [ ]:
plt.imshow(ax.get_array())

In [ ]:
ax.write_png("test.png")

In [ ]:
vis.get_pylab_image(ax)

In [ ]:
x,y=dataset.load_obj_id("02691156_131db4a650873babad3ab188d086d4db")

In [ ]:
vis.sample(x,y,y)

In [ ]:
"02691156_131db4a650873babad3ab188d086d4db"

In [ ]:
vis.voxel_binary(yp[0],"frame_{}".format(i))
image_list=utils.filter_files("frame*.png")
print(image_list)
clip=ImageSequenceClip(image_list,fps=6)
clip.write_videofile("movie.mp4")


In [ ]:
loss_paths = dataset.construct_file_path_list_from_dir("aws", "loss.npy")
for path in loss_paths:
    print(path)
    print(utils.grep_learning_rate(path))
    loss_arr = np.load(path)
    print(loss_arr.shape)
    plt.plot(loss_arr.flat)
    plt.show()

In [ ]:
p=np.load("voxel_prediction.npy")
p=np.transpose(p,[0,1,3,4,2])

In [ ]:
vp=np.load("voxel_prediction.npy")[23]
vp=np.transpose(vp,[0,2,3,1])
vis.voxel_b(vp)

In [ ]:
vis.voxel_binary(yp[0])
plt.figure()
plt.imshow(vis.voxel_ndarray(yp[0]))

In [ ]:
weights=np.load("/Users/micmelesse/Documents/3D-reconstruction-with-neural-networks/data/ResidualGRUNet.npy")

In [ ]:
l=utils.list_folders("aws")[-1]
png_ls=dataset.construct_file_path_list_from_dir("aws/model_2018-04-02_20:10:12_L:0.1_E:40_B:16", "_p.png")

In [ ]:
png_ls

In [ ]:
for f in png_ls:
    obj_id=utils.grep_obj_id(f)
    n=utils.grep_stepcount(f)
    new_name=os.path.dirname(f)+"/"+n+'_'+obj_id+'_yp.png'
#     print(f,new_name)
    os.rename(f,new_name)
    

In [ ]:
png_ls

In [ ]:
Image.open(png_ls[2])

In [ ]:
utils.grep_obj_id("uiudasniuansfuas_asnfasnfkasfunkasufn_x.png")

In [ ]:
# toy example

import tensorflow as tf

LEARNING_RATE, EPOCHS=0.01,100                              # hyper paramters

# Build Tensorflow Graph
X = tf.constant([[1],[2],[3]],dtype=tf.float32)             # input
W = tf.Variable(tf.random_uniform([3,3]),dtype=tf.float32)  # paramters
Y = tf.matmul(W,X)                                          # prediction
T = tf.constant([[3],[2],[1]],dtype=tf.float32)             # target label
E = tf.reduce_mean(tf.abs(T-Y))                             # Error 
GD=tf.train.GradientDescentOptimizer(LEARNING_RATE)         # optimizing algorihtm
Gradients = GD.compute_gradients(E)                         # compute the gradients from the Error
TRAIN_STEP = GD.apply_gradients(Gradients)                  # use the gradients to update their paramters


# Run Graph
with tf.Session() as sess:                                  # start tensorflow session
    tf.global_variables_initializer().run()                 # initialize variables
    for i in range(EPOCHS):                                 # open pass over the dataset
        result=sess.run([E,Y,TRAIN_STEP])                   # fetch values from tensorflow graph
        
print(result[0]) 
print(result[1])

In [ ]:
kernel=tf.random_uniform([3,3,10,10])
print(kernel)

kern_1 = tf.concat(tf.unstack(kernel, axis=-1), axis=-1)
kern_2 = tf.transpose(kern_1, [2,0,1])
kern_3 = tf.expand_dims(kern_2, -1)

print(kern_1)
print(kern_2)
print(kern_3)

In [ ]:
ret=tf.random_uniform([36,5,127,127,3])
print(ret)

feature_map_1 = tf.concat(tf.unstack(ret, axis=4), axis=2)
feature_map_2 = tf.concat(tf.unstack(feature_map_1, axis=1), axis=2)
feature_map_3 = tf.expand_dims(feature_map_2, -1)

print(feature_map_1)
print(feature_map_2)
print(feature_map_3)

In [ ]:
kernel=tf.random_uniform([3,3,3,10,10])
print(kernel)

kern_1 = tf.concat(tf.unstack(kernel, axis=-1), axis=-1)
kern_2 = tf.transpose(kern_1,[3,0,1,2])
kern_3 = tf.expand_dims(kern_2,-1)
kern_4 = tf.concat(tf.unstack(kern_3, axis=1), axis=1)

print(kern_1)
print(kern_2)
print(kern_3)
print(kern_4)

In [ ]:
ret=tf.random_uniform([2,32,32,32,2])
print(ret)

vox_slice_1 = tf.unstack(ret, axis=4)[1]
vox_slice_2 = tf.split(vox_slice_1,4, axis=3)
vox_slice_3 = tf.concat(vox_slice_2, axis=2)
vox_slice_4 = tf.concat(tf.unstack(vox_slice_3,axis=-1),axis=1)
vox_slice_5 = tf.expand_dims(vox_slice_4,-1)

print(vox_slice_1)
print(vox_slice_2)
print(vox_slice_3)
print(vox_slice_4)
print(vox_slice_5)


In [ ]:
import tensorflow as tf
LEARNING_RATE, EPOCHS = 0.01,100                     # hyper-parameters

# Build Tensorflow Graph
X = tf.constant([[1.0],[2.0],[3.0]])                 # input
T = tf.constant([[3.0],[2.0],[1.0]])                 # target
W = tf.Variable(tf.random_uniform([3,3]))            # parameters
Y = tf.matmul(W,X)                                   # prediction
L = tf.reduce_mean(tf.abs(T-Y))                      # loss(error)

GD = tf.train.GradientDescentOptimizer(LEARNING_RATE)# optimizing algorithm
Gradients = GD.compute_gradients(L)                  # gradient of the error function
TRAIN_STEP = GD.apply_gradients(Gradients)           # update parameters using gradients

# Run Graph
with tf.Session() as sess:                           # start tensorflow session
    tf.global_variables_initializer().run()          # initialize variables
    for i in range(EPOCHS):                          # iterate over the dataset EPOCHS times
        result=sess.run([L,Y,TRAIN_STEP])            # fetch values from graph
        
print("Loss:", result[0]) 
print("Prediction:", result[1])